# Import Azure ML Labeling Tags to Custom Vision Service

In [ ]:
# install packages if needed
import sys
!{sys.executable} -m pip install azure-cognitiveservices-vision-customvision
!{sys.executable} -m pip install azureml-sdk
!{sys.executable} -m pip install azureml-contrib-dataset

In [1]:
import json, os, shutil, math

import azureml.contrib.dataset
from azureml.core import Workspace, Dataset, Datastore
from azureml.contrib.dataset import FileHandlingOption

from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials

# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
# azureml-contrib-dataset of version 1.0.72 or higher is required

## 1. Set up Custom Vision project

Enter the details for your Custom Vision endpoint and training key below:

In [2]:
ENDPOINT = 'https://YOUR_REGION.api.cognitive.microsoft.com'
training_key = "<CUSTOM VISION TRAINING KEY>"

credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(endpoint=ENDPOINT, credentials=credentials)

### Option 1: Create new project

The below cell will create a new Custom Vision project.  Enter a name for your project below:

Note: If you have an existing project, skip to the next cell.

In [3]:
project_name = "<PROJECT NAME>"

# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Create a new project
print("Creating new project...")
project = trainer.create_project(project_name, domain_id=obj_detection_domain.id)
print(project.name, "project created")

Creating new project...
Sample project created


### Option 2: Update existing Custom Vision project

The below cell will get an existing project by ID.  Enter your project ID below (you can retrieve this ID from your project in the [Custom Vision portal](http://customvision.ai)):

In [ ]:
project_id = "<PROJECT ID>"

# Get existing project
project = trainer.get_project(project_id = project_id) 
print(project.name, "project retrieved")

## 2. Get labeled dataset from Azure ML

After labeling images with Azure Machine Learning, you can export the tags as an *Azure ML Dataset*:

<img align="left" src="../assets/aml_label_export.png">

Retrieve the resulting dataset name from your *Datasets* in Azure Machine Learning Studio, and enter it below.  Similarly, enter the details for your subscription, resource group, and workspace.  You can also retrieve your Azure ML workspace through a [workspace config.json file](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-environment#workspace)

In [5]:
subscription_id = '<SUBSCRIPTION ID>'
resource_group = '<RESOURCE GROUP>'
workspace_name = '<AML WORKSPACE NAME>'
dataset_name = '<LABELED DATASET NAME>'

ws = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(ws, name=dataset_name)
df = dataset.to_pandas_dataframe(FileHandlingOption.DOWNLOAD, target_path='/tmp', overwrite_download=True)
df.head()

,image_url,label,label_confidence
0,/tmp/workspaceblobstore/UI/04-17-2020_033330_U...,"[{'label': 'building', 'topX': 0.2188137755102...","[1.0, 1.0, 1.0, 1.0]"
1,/tmp/workspaceblobstore/UI/04-17-2020_033330_U...,"[{'label': 'vehicle', 'topX': 0.37935799319727...","[1.0, 1.0, 1.0, 1.0, 1.0]"
2,/tmp/workspaceblobstore/UI/04-17-2020_033330_U...,"[{'label': 'vehicle', 'topX': 0.28434311224489...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]"


## 3. Prep images and format tags for Custom Vision

Below, we format the labels from our Azure ML labeled dataset into regions that can be uploaded to the Custom Vision service with their corresponding images.

In [6]:
tagged_ims = []
tags = trainer.get_tags(project.id)

for i, img in df.iterrows():
    filename = img['image_url']
    labels = img['label']
    regions = []
    
    # parse labels
    for label in labels:
        label_name = label['label']

        l = label['topX']
        t = label['topY']
        r = label['bottomX']
        b = label['bottomY']

        w = r-l
        h = b-t
        
        # retrieve tag object by label name
        try:
            index = [x.name for x in tags].index(label_name)
            tag = tags[index]
        # create tag if it does not exist yet
        except:
            print("Creating new tag for:", label_name)
            tag = trainer.create_tag(project.id, label_name)
            tags = trainer.get_tags(project.id)

        # create bounding box regions
        regions.append(Region(tag_id=tag.id,left=l,top=t,width=w,height=h))

        with open(filename, mode="rb") as im_data:
            tagged_ims.append(ImageFileCreateEntry(name=filename, contents=im_data.read(), regions=regions))

Creating new tag for: building
Creating new tag for: vehicle


## 4. Upload images and tags to Custom Vision

Upload images and labels in batches of 64:

In [7]:
print("Uploading images and tags")
for i in range(0, len(tagged_ims), 64):   
    batch = tagged_ims[i:i+64]
    trainer.create_images_from_files(project.id, images=batch)

Uploading images and tags
